In [217]:
class convertCVtoText:
    @staticmethod
    def startConversion(fileName):
        import PyPDF2
        pdfFileObj = open(fileName,'rb')     #'rb' for read binary mode
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        noOfPages=pdfReader.numPages
        text=""
        for pages in range(0,pdfReader.numPages):
            pageObj = pdfReader.getPage(pages)          #'9' is the page number
            text+=pageObj.extractText()
        return text

In [218]:
class regularExp:
    @staticmethod
    def normalizeText(text):
        norText=""
        import re
        norText+= re.sub(r'[^a-zA-Z ]',r' ',text)
        return norText
    @staticmethod
    def Stemmer(text):
        from nltk.stem import PorterStemmer
        import nltk
        words=nltk.word_tokenize(text)
        stemmer = PorterStemmer()
        singles = [stemmer.stem(word) for word in words]
        #print(singles)
        singles=list(set(singles))
        #print(singles)
        stemmedText=" ".join(singles)
        #print(stemmedText)
        return stemmedText
        
        

In [219]:
class NLTKHelper:
    @staticmethod
    def findDocumentMatrix(totalCVText,minFrequency):
        from sklearn.feature_extraction.text import CountVectorizer
        vectorizer=CountVectorizer(stop_words='english',min_df=minFrequency)
        documentMatrix=vectorizer.fit_transform(totalCVText)
        vocabulary=vectorizer.vocabulary_
        return documentMatrix,vocabulary     

In [220]:
class CV:
     def __init__(self,path):
            self.fileName=path
            self.text=convertCVtoText.startConversion(self.fileName)
            #self.stemedText=regularExp.Stemmer(self.text)
            self.cleanText=regularExp.normalizeText(self.text)
        


In [221]:
class databaseHandler:
    def __init__(self,password="ST.xavier51",databaseName="cvRanking"):
        import pymysql
        try:
            self.database=pymysql.connect("localhost","root","ST.xavier51","cvRanking")
            self.cursor = self.database.cursor()
        except Exception as e:
            print("error info %s"%e)
    def entryTheCV(self,data_CV):
        try:
#             dataReturned=self.checkDuplication(data_CV[1])
#             if(len(dataReturned)==0):
                add_CV="""insert into recordentry
                    (checkRecord,recordname)
                    values(%s,%s)"""
    #             add_CV="""insert into recordentry
    #                 (idrecordEntry,checkRecord,recordname)
    #                 values(%s,%s,%s)"""
                #data_CV=("yes","ram")
                print(data_CV)
                self.cursor.execute(add_CV,data_CV)
                self.database.commit()
                print("CV name is entered in database")
        except Exception as e:
            print("in entryTheCV\t"+str(e))
    def checkDuplication(self,name_CV):
        try:
            check_CV="""select* from recordentry as re where re.recordname=%s"""
            self.cursor.execute(check_CV,name_CV)
            result = self.cursor.fetchall()
        except Exception as e:
            print("in process checkDuplication\t"+str(e))
        if(len(result)==0):  
            return False
        else:
            return True  
    def closeConnection(self):
        self.database.close()


In [222]:
class CVManager:
    def __init__(self):
        self.newCVAdded=False
        self.database=databaseHandler()
        self.CVList=[]
        self.CVFileName=[]
        self.fileNamesWithPath=[]
        self.CVTextColl=[]
        self.documentMatrix=[]
        self.vocabulary={}
    def list_CVs(self,rootPath):
        import os
        for root, dirs, files in os.walk(rootPath):
            for name in files:
                self.CVFileName.append(name)
                self.fileNamesWithPath.append(os.path.join(root, name))
    def collectCV(self):
        for (cvPath,cvName) in zip(self.fileNamesWithPath,self.CVFileName):
            try:
                if(not(self.database.checkDuplication(cvName))):
                    newCV=CV(cvPath)
                    self.CVList.append(newCV)
                    self.database.entryTheCV(("yes",cvName))
                    self.newCVAdded=True
                else:
                    newCV=CV(cvPath)
                    self.CVList.append(newCV)
                    print("%s named CV is already in database"%cvName)
            except Exception as e:
                print("in collection of CV \t"+str(e))
    def collectCVText(self):
        for cv in self.CVList:
            self.CVTextColl.append(cv.cleanText) 
    def findDocumentMatrix(self,minFrequency):
        import json
        try:
            if(self.newCVAdded):
                self.collectCVText()
                print("here")
                self.documentMatrix,self.vocabulary=NLTKHelper.findDocumentMatrix(self.CVTextColl,minFrequency)
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVText.json","w") as file:
                    json.dump(self.CVTextColl,file)
                file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\documentMatrix.txt","w") as file:
#                     json.dump(list(self.documentMatrix),file)
#                 file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\vocabulary.txt","w") as file:
#                     json.dump(self.vocabulary,file)
#                 file.close()
                #print( self.vocabulary)
            else:
                with open("C:\\Users\\sagar\\Desktop\\CV coll\\wholeCVText.txt","r") as file:
                    self.CVTextColl=json.load(file)
                file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\documentMatrix.txt","r") as file:
#                     json.loads(self.documentMatrix,file)
#                 file.close()
#                 with open("C:\\Users\\sagar\\Desktop\\CV coll\\vocabulary.txt","r") as file:
#                     json.loads(self.vocabulary,file)
#                 file.close()
        except Exception as e:
                print("in reading/writing of saved files \t"+str(e))
                
                
            
        
        

In [223]:
manager=CVManager()

In [224]:
manager.list_CVs("C:\\Users\\sagar\\Desktop\\CV coll\\kailesh\\creative director")

In [225]:
#manager.CVFileName

In [226]:
manager.collectCV()

Agnes-Szelag.pdf named CV is already in database
Alison-Haller.pdf named CV is already in database
Brad-Blackburn.pdf named CV is already in database
Bruce-Jones.pdf named CV is already in database
Carolyn-Carmel.pdf named CV is already in database
Christina-Le-Sesne.pdf named CV is already in database
Christopher-Mackey.pdf named CV is already in database
Cosmo-Nardozza.pdf named CV is already in database
Craig-Thacker.pdf named CV is already in database
Cristopher-Schafer.pdf named CV is already in database
Daniel-Montgomery.pdf named CV is already in database
David-Cordon.pdf named CV is already in database
David-Hindelang.pdf named CV is already in database
David-Hunter.pdf named CV is already in database
David-Moshier.pdf named CV is already in database
Denise-Sucik.pdf named CV is already in database
Destiny-Washington.pdf named CV is already in database
Devan-Brush.pdf named CV is already in database
Eric-Bronk.pdf named CV is already in database
Hannah-Nichols.pdf named CV is a

In [227]:
manager.findDocumentMatrix(20)


In [228]:
manager.CVTextColl

['Agnes Szelag Creative Director Oakland  CA   Email me on Indeed   indeed com r Agnes Szelag e dc ffa  a      Authorized to work in the US for any employer WORK EXPERIENCE Creative Director Berkeley Resource Center for Online Education  UC Berkeley        to        Direct all media and art production  concept  pre production  storyboards  studio and on location production  graphic design  motion graphics  post production  and distribution   With management team create and manage the vision  work culture  mission statement  operational structure and workflows for production team consisting of instructional designers  product managers  and media team    Lead media team consisting of  editors  camera people  motion graphic artists  animators  and graphic designers  Occasionally working with contractors freelancers    Lead concept brainstorming sessions for art and story development    Work with Product Managers and clients to create the most appropriate and effective media for each cours